In [13]:
import os
import pandas as pd
import tqdm
from multiprocessing import Pool
from rdkit.Chem import MolFromSmiles, MolToSmiles

In [2]:
filepath = os.path.join('data', 'gdb13.1M.freq.ll.smi')

In [8]:
df_raw = pd.read_csv(filepath, header=None, sep='\t')

print(df_raw.shape)
df_raw.head()

(1000000, 3)


,0,1,2
0,C1=Cc2cc1nnc1snc(o2)-o-1,0,68.182535
1,N1C2C3C4C5NC6C7C6C5(C13)C2N47,0,67.352869
2,c1c2c[nH]c(nn3cnc(c#1)c3)-s-2,0,65.054106
3,N=c1-c2cnn-1cnccc(=O)c2,0,62.522982
4,C=Nn1-c2cccconc-1[nH]c2,0,59.586299


In [14]:
def normalize(smi):
    can = MolToSmiles(MolFromSmiles(smi), kekuleSmiles=True)
    lgt = len(can)
    return can, lgt

In [15]:
def loader(df):
    for i in tqdm.trange(len(df)):
        yield df_raw.iloc[i,0]

In [17]:
with Pool(30) as p:
    records = p.map(normalize, loader(df_raw))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000000/1000000 [00:05<00:00, 179849.38it/s]


In [19]:
df = pd.DataFrame.from_records(records)
df = df.rename(columns={0:'smiles', 1:'length'})

In [20]:
print(df.shape)
df.head()

(1000000, 2)


,smiles,length
0,C1=CC2=NN=C3OC(=NS3)OC1=C2,26
1,N1C2C3C4C5NC6C7C6C5(C13)C2N47,29
2,C1#CC2=CN(C=N2)N=C2NC=C1S2,26
3,N=C1C2=CC(=O)C=CN=CN1N=C2,25
4,C=NN1C2=CNC1=NOC=CC=C2,22


In [21]:
df.to_csv(os.path.join('data', 'gdb13.csv'), index=False)